## Team members
### 1. Chetan Naveen Vadde
### 2. Vishal Singh
### 3. Reshma Punukollu

## Join 'plane-data.csv' to 2 'flight' files (2007.csv and 2008.csv) to 'airport-codes-na.txt'

###1) Remove any NULL rows from 'plane-data.csv'
###2) Write most efficient code for query below:
###  Query: SELECT 'UniqueCarrier', 'manufacturer', 'model', 'Country', 'IATA', count of 'DepDelay'
###  GROUP BY non-Aggregated columns
###  WHERE 'DepDelay' > 180 minutes and UniqueCarrier = 'WN' 
###  and Manufacturer is any 'Airbus' or 'Boeing' and 'Country' = 'USA' 
###  ORDER BY 'DepDelay' Descending

### Note:  You can only use 1 WHERE clause in any of the 3 DataFrames (planesDF, flightsDF, airportsDF) of your choice
### Use other means to skip Data for other WHERE columns or elsewhere (Cache, Predicate Pushdown, ZOrder, Bloom Filters, etc.)
### You cannot use same Performance Technique more than once in Hackathon (for example, can Cache 1 DataFrame, but not multiple)
### You will be graded on: How many different Perf Tuning techinques you used and Overall Clock time of final query

## Lab 01: First check out the Size (in bytes) of the Data

In [0]:
# Here's the 'planesDF' data
display(dbutils.fs.ls("dbfs:/databricks-datasets/asa/planes/plane-data.csv"))

path,name,size,modificationTime
dbfs:/databricks-datasets/asa/planes/plane-data.csv,plane-data.csv,428796,1459247088000


In [0]:
# Here's the 'flightsDF' data
display(dbutils.fs.ls("dbfs:/databricks-datasets/asa/airlines/2007.csv"))
display(dbutils.fs.ls("dbfs:/databricks-datasets/asa/airlines/2008.csv"))

path,name,size,modificationTime
dbfs:/databricks-datasets/asa/airlines/2007.csv,2007.csv,702878193,1459245210000


path,name,size,modificationTime
dbfs:/databricks-datasets/asa/airlines/2008.csv,2008.csv,689413344,1459246170000


In [0]:
# Here's the 'airportsDF' data
display(dbutils.fs.ls("dbfs:/databricks-datasets/flights/airport-codes-na.txt"))

path,name,size,modificationTime
dbfs:/databricks-datasets/flights/airport-codes-na.txt,airport-codes-na.txt,11411,1457749605000


### Lab 02: Check out what data looks like and if it has Headers

In [0]:
dbutils.fs.head("dbfs:/databricks-datasets/asa/planes/plane-data.csv")

[Truncated to first 65536 bytes]
Out[4]: 'tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year\nN050AA\nN051AA\nN052AA\nN054AA\nN055AA\nN056AA\nN057AA\nN058AA\nN059AA\nN060AA\nN061AA\nN062AA\nN063AA\nN064AA\nN065AA\nN066AA\nN067AA\nN068AA\nN069AA\nN070AA\nN071AA\nN072AA\nN073AA\nN074AA\nN075AA\nN076AA\nN077AA\nN078AA\nN079AA\nN080AA\nN081AA\nN082AA\nN083AA\nN084AA\nN10156,Corporation,EMBRAER,02/13/2004,EMB-145XR,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2004\nN102UW,Corporation,AIRBUS INDUSTRIE,05/26/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998\nN10323,Corporation,BOEING,07/01/1997,737-3TO,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1986\nN103US,Corporation,AIRBUS INDUSTRIE,06/18/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999\nN104UA,Corporation,BOEING,01/26/1998,747-422,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998\nN104UW,Corporation,AIRBUS INDUSTRIE,07/02/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999\nN10575,Corporation,EMB

In [0]:
dbutils.fs.head("dbfs:/databricks-datasets/asa/airlines/2007.csv")

[Truncated to first 65536 bytes]
Out[5]: 'Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay\n2007,1,1,1,1232,1225,1341,1340,WN,2891,N351,69,75,54,1,7,SMF,ONT,389,4,11,0,,0,0,0,0,0,0\n2007,1,1,1,1918,1905,2043,2035,WN,462,N370,85,90,74,8,13,SMF,PDX,479,5,6,0,,0,0,0,0,0,0\n2007,1,1,1,2206,2130,2334,2300,WN,1229,N685,88,90,73,34,36,SMF,PDX,479,6,9,0,,0,3,0,0,0,31\n2007,1,1,1,1230,1200,1356,1330,WN,1355,N364,86,90,75,26,30,SMF,PDX,479,3,8,0,,0,23,0,0,0,3\n2007,1,1,1,831,830,957,1000,WN,2278,N480,86,90,74,-3,1,SMF,PDX,479,3,9,0,,0,0,0,0,0,0\n2007,1,1,1,1430,1420,1553,1550,WN,2386,N611SW,83,90,74,3,10,SMF,PDX,479,2,7,0,,0,0,0,0,0,0\n2007,1,1,1,1936,1840,2217,2130,WN,409,N482,101,110,89,47,56,SMF,PHX,647,5,7,0,,0,46,0,0,0,1\n2007,1,1,1,944,935,122

In [0]:
dbutils.fs.head("dbfs:/databricks-datasets/asa/airlines/2008.csv")

[Truncated to first 65536 bytes]
Out[6]: 'Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay\n2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA\n2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA\n2008,1,3,4,628,620,804,750,WN,448,N428WN,96,90,76,14,8,IND,BWI,515,3,17,0,,0,NA,NA,NA,NA,NA\n2008,1,3,4,926,930,1054,1100,WN,1746,N612SW,88,90,78,-6,-4,IND,BWI,515,3,7,0,,0,NA,NA,NA,NA,NA\n2008,1,3,4,1829,1755,1959,1925,WN,3920,N464WN,90,90,77,34,34,IND,BWI,515,3,10,0,,0,2,0,0,0,32\n2008,1,3,4,1940,1915,2121,2110,WN,378,N726SW,101,115,87,11,25,IND,JAX,688,4,10,0,,0,NA,NA,NA,NA,NA\n2008,1,3,4,1937,1830,2037,1940,WN,509,N763SW,240,250,230,57,67,IND,LAS,159

In [0]:
dbutils.fs.head("dbfs:/databricks-datasets/flights/airport-codes-na.txt")

Out[7]: 'City\tState\tCountry\tIATA\rAbbotsford\tBC\tCanada\tYXX\rAberdeen\tSD\tUSA\tABR\rAbilene\tTX\tUSA\tABI\rAkron\tOH\tUSA\tCAK\rAlamosa\tCO\tUSA\tALS\rAlbany\tGA\tUSA\tABY\rAlbany\tNY\tUSA\tALB\rAlbuquerque\tNM\tUSA\tABQ\rAlexandria\tLA\tUSA\tAEX\rAllentown\tPA\tUSA\tABE\rAlliance\tNE\tUSA\tAIA\rAlpena\tMI\tUSA\tAPN\rAltoona\tPA\tUSA\tAOO\rAmarillo\tTX\tUSA\tAMA\rAnahim Lake\tBC\tCanada\tYAA\rAnchorage\tAK\tUSA\tANC\rAppleton\tWI\tUSA\tATW\rArviat\tNWT\tCanada\tYEK\rAsheville\tNC\tUSA\tAVL\rAspen\tCO\tUSA\tASE\rAthens\tGA\tUSA\tAHN\rAtlanta\tGA\tUSA\tATL\rAtlantic City\tNJ\tUSA\tACY\rAugusta\tGA\tUSA\tAGS\rAugusta\tME\tUSA\tAUG\rAustin\tTX\tUSA\tAUS\rBagotville\tPQ\tCanada\tYBG\rBaie-Comeau\tPQ\tCanada\tYBC\rBakersfield\tCA\tUSA\tBFL\rBaltimore\tMD\tUSA\tBWI\rBangor\tME\tUSA\tBGR\rBar Harbor\tME\tUSA\tBHB\rBarrow\tAK\tUSA\tBRW\rBaton Rouge\tLA\tUSA\tBTR\rBeaumont\tTX\tUSA\tBPT\rBeckley\tWV\tUSA\tBKW\rBedford\tMA\tUSA\tBED\rBellingham\tWA\tUSA\tBLI\rBemidji\tMN\tUSA\tBJI\rBethel\t

### Lab 03: Code in Cells below
1) Remove any NULL rows from 'plane-data.csv'
2) Write most efficient code for query below:
  Query: SELECT 'UniqueCarrier', 'manufacturer', 'model', 'Country', 'IATA', count of 'DepDelay'
  GROUP BY non-Aggregated columns
  WHERE 'DepDelay' > 180 minutes and UniqueCarrier = 'WN' 
  and Manufacturer is any 'Airbus' or 'Boeing' and 'Country' = 'USA' 
  ORDER BY 'DepDelay' Descending

 Note:  You can only use 1 WHERE clause in any of the 3 DataFrames (planesDF, flightsDF, airportsDF) of your choice.
 Use other means to skip Data for other WHERE columns or elsewhere (Cache, Predicate Pushdown, ZOrder, Bloom Filters, etc.)
 You cannot use same Performance Technique more than once in Hackathon (for example, can Cache 1 DataFrame, but not multiple)
 You will be graded on: How many different Perf Tuning techinques you used and Overall Clock time of final query

In [0]:
import pyspark.sql.functions as F


In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", True)

In [0]:
planes_df = spark.read.options(header='True', inferSchema='True').csv("dbfs:/databricks-datasets/asa/planes/plane-data.csv").dropna().where(F.col("manufacturer").isin(['AIRBUS INDUSTRIE','BOEING'])).select("manufacturer","tailnum","model")

In [0]:
planes_df.show()

+----------------+-------+--------+
|    manufacturer|tailnum|   model|
+----------------+-------+--------+
|AIRBUS INDUSTRIE| N102UW|A320-214|
|          BOEING| N10323| 737-3TO|
|AIRBUS INDUSTRIE| N103US|A320-214|
|          BOEING| N104UA| 747-422|
|AIRBUS INDUSTRIE| N104UW|A320-214|
|          BOEING| N105UA| 747-451|
|AIRBUS INDUSTRIE| N105UW|A320-214|
|AIRBUS INDUSTRIE| N106US|A320-214|
|          BOEING| N107UA| 747-422|
|AIRBUS INDUSTRIE| N107US|A320-214|
|AIRBUS INDUSTRIE| N108UW|A320-214|
|AIRBUS INDUSTRIE| N109UW|A320-214|
|AIRBUS INDUSTRIE| N110UW|A320-214|
|AIRBUS INDUSTRIE| N111US|A320-214|
|          BOEING| N11206| 737-824|
|AIRBUS INDUSTRIE| N112US|A320-214|
|AIRBUS INDUSTRIE| N113UW|A320-214|
|AIRBUS INDUSTRIE| N114UW|A320-214|
|          BOEING| N11612| 737-524|
|          BOEING| N11641| 737-524|
+----------------+-------+--------+
only showing top 20 rows



In [0]:
flights_df = spark.read.options(header='True', inferSchema='True').csv(["dbfs:/databricks-datasets/asa/airlines/2007.csv","dbfs:/databricks-datasets/asa/airlines/2008.csv"]).select("UniqueCarrier","TailNum","Origin","Dest","DepDelay").filter(F.col("DepDelay")>180).filter(F.col("UniqueCarrier")=="WN")
flights_df.show()

+-------------+-------+------+----+--------+
|UniqueCarrier|TailNum|Origin|Dest|DepDelay|
+-------------+-------+------+----+--------+
|           WN|   N765|   CMH| MDW|     229|
|           WN|   N789|   BNA| PHL|     181|
|           WN| N304SW|   GEG| PDX|     193|
|           WN|   N431|   LAS| GEG|     196|
|           WN|   N478|   MCO| PHL|     188|
|           WN|   N789|   MCO| PHL|     185|
|           WN|   N225|   MDW| PHL|     225|
|           WN| N615SW|   MHT| PHL|     196|
|           WN| N697SW|   PDX| GEG|     223|
|           WN| N304SW|   PDX| GEG|     183|
|           WN| N304SW|   PDX| SMF|     192|
|           WN|   N789|   PHL| BNA|     198|
|           WN|   N225|   PHL| LAS|     252|
|           WN|   N215|   PHL| OAK|     195|
|           WN|   N370|   PHL| PIT|     182|
|           WN| N615SW|   PHL| RDU|     229|
|           WN| N328SW|   PHL| RDU|     182|
|           WN|   N478|   PHL| TPA|     196|
|           WN|   N311|   PHX| STL|     195|
|         

In [0]:
airports_df = spark.read.option("delimiter", "\t").option("inferSchema","true").load("dbfs:/databricks-datasets/flights/airport-codes-na.txt", 
                       format="csv", header=True).filter(F.col('Country') == 'USA').select("IATA","Country")
airports_df.show()

+----+-------+
|IATA|Country|
+----+-------+
| ABR|    USA|
| ABI|    USA|
| CAK|    USA|
| ALS|    USA|
| ABY|    USA|
| ALB|    USA|
| ABQ|    USA|
| AEX|    USA|
| ABE|    USA|
| AIA|    USA|
| APN|    USA|
| AOO|    USA|
| AMA|    USA|
| ANC|    USA|
| ATW|    USA|
| AVL|    USA|
| ASE|    USA|
| AHN|    USA|
| ATL|    USA|
| ACY|    USA|
+----+-------+
only showing top 20 rows



In [0]:
joined_df = flights_df.join(planes_df,flights_df.TailNum ==  planes_df.tailnum,"inner")
# Using (joined_df.Origin == airports_df.IATA) | (joined_df.Dest == airports_df.IATA) as joined condition because it wasn't clearly mentioned which columns to join on and either origin or destination could be used as IATA. 
# This would cause the run time to increase but we thought it might be consider than use either Origin or destination
joined_df = joined_df.join(F.broadcast(airports_df),(joined_df.Origin == airports_df.IATA) | (joined_df.Dest == airports_df.IATA),"inner")
joined_df = joined_df.drop("Dest","Origin","tailnum")
joined_df.cache()
joined_df.count()

Out[7]: 20838

In [0]:
joined_df.show()

+-------------+--------+------------+-------+----+-------+
|UniqueCarrier|DepDelay|manufacturer|  model|IATA|Country|
+-------------+--------+------------+-------+----+-------+
|           WN|     193|      BOEING|737-3H4| PDX|    USA|
|           WN|     193|      BOEING|737-3H4| GEG|    USA|
|           WN|     196|      BOEING|737-3H4| MHT|    USA|
|           WN|     196|      BOEING|737-3H4| PHL|    USA|
|           WN|     223|      BOEING|737-3TO| PDX|    USA|
|           WN|     223|      BOEING|737-3TO| GEG|    USA|
|           WN|     183|      BOEING|737-3H4| PDX|    USA|
|           WN|     183|      BOEING|737-3H4| GEG|    USA|
|           WN|     192|      BOEING|737-3H4| PDX|    USA|
|           WN|     192|      BOEING|737-3H4| SMF|    USA|
|           WN|     229|      BOEING|737-3H4| PHL|    USA|
|           WN|     229|      BOEING|737-3H4| RDU|    USA|
|           WN|     182|      BOEING|737-3H4| PHL|    USA|
|           WN|     182|      BOEING|737-3H4| RDU|    US

In [0]:
joined_df = joined_df.groupBy( "UniqueCarrier","Country",'manufacturer', 'model', 'IATA').count().withColumn("DepDelayCount",F.col("count")).drop("count")
joined_df.orderBy("DepDelayCount", ascending=False).show()

+-------------+-------+------------+-------+----+-------------+
|UniqueCarrier|Country|manufacturer|  model|IATA|DepDelayCount|
+-------------+-------+------------+-------+----+-------------+
|           WN|    USA|      BOEING|737-7H4| MDW|         1405|
|           WN|    USA|      BOEING|737-7H4| PHL|          796|
|           WN|    USA|      BOEING|737-7H4| BWI|          778|
|           WN|    USA|      BOEING|737-7H4| LAS|          752|
|           WN|    USA|      BOEING|737-3H4| BWI|          518|
|           WN|    USA|      BOEING|737-3H4| MDW|          480|
|           WN|    USA|      BOEING|737-7H4| PHX|          476|
|           WN|    USA|      BOEING|737-3H4| PHL|          439|
|           WN|    USA|      BOEING|737-3H4| DAL|          412|
|           WN|    USA|      BOEING|737-3H4| HOU|          376|
|           WN|    USA|      BOEING|737-7H4| MCO|          361|
|           WN|    USA|      BOEING|737-7H4| HOU|          340|
|           WN|    USA|      BOEING|737-

In [0]:
joined_df.count()

Out[10]: 699

# Inside this Cell, tell me which Performance Techniques you used on the DataFrames or within the query
# I would expect at least 4 Techniques at a minimum.
### We used the following performance tuning techinques
#### 1. Late push filter - predicate pushdown
#### 2. Columnar Pushdown - predicate pushdown
#### 3. Cached the dataframe which could be used multiple times
#### 4. we used broadcast join for better peformance during a join

# Step 4: Upload this Notebook to Canvas for grading
## Deadline for email submission 3 days from now
## To save this Notebook as file, do the following:
  
#####1) In gray vertical bar on left, click on 'Workspace' icon
#####2) Find notebook, and click on 'down' arrow on right side of it
#####3) Select 'Export' > 'IPython Notebook'
#####4) Click 'Save'
#####5) Upload to Canvas for grading
#####   along with names of any team members so I can assign grades